<a href="https://colab.research.google.com/github/nalrob/Datos_Masivos_MCD/blob/main/Practica_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practica 1: Paralelizacion.

## Parte 1: Investigar sobre las lazy variables y como se usa el Delay para problemas mas grandes.


"Lazy Variable" o variable perezosa es una variable que se inicializa solo cuando se llama por primera vez.
Esta técnica se utiliza para mejorar el rendimiento y la eficiencia de los programas, ya que puede ahorrar tiempo y recursos al no calcular o inicializar valores que nunca se utilizarán.


Ejemplo de su uso:

In [ ]:
from functools import lru_cache

class Person:
    def __init__(self, weight, height):
        self.weight = weight
        self.height = height

    @property
    @lru_cache(maxsize=None)
    def BMIIndex(self):
        print("Calculando el índice de masa corporal...")
        return self.weight / pow(self.height, 2)

# Crear una instancia de la clase Person
person = Person(70.0, 1.75)

# Acceder a la propiedad perezosa BMIIndex
print(person.BMIIndex)


En programación, existen diferentes tipos de variables "lazy" o perezosas, que se utilizan en diferentes contextos y para diferentes propósitos. Algunos de los tipos más comunes de variables "lazy" son:

**1.   Lazy evaluation**
 
> Se utiliza en lenguajes de programación que tienen una evaluación perezosa, donde las expresiones no se evalúan hasta que se utilizan. Esto significa que los valores solo se calculan cuando es necesario, lo que puede mejorar el rendimiento y la eficiencia del programa.



**2.   Lazy initialization**


> Se utiliza para retrasar la inicialización de una variable hasta que sea necesaria. Por ejemplo, una variable que se utiliza raramente puede inicializarse perezosamente para evitar la sobrecarga de la inicialización al inicio del programa.




**3.    Lazy loading**


> Se utiliza para cargar datos o recursos solo cuando son necesarios. Por ejemplo, en una aplicación web, se pueden cargar imágenes o archivos solo cuando se muestran en la página web, lo que mejora el rendimiento y la 


**4.   Memorization**

> Se utiliza para almacenar el resultado de una función costosa en caché y devolver el resultado almacenado en lugar de volver a calcularlo cuando se llama a la función con los mismos parámetros. Esto mejora significativamente el rendimiento de la función en llamadas repetidas con los mismos parámetros.


**5.   Thunk**


> Es una función que se utiliza para retrasar la evaluación de una expresión hasta que sea necesaria. El término "thunk" proviene de la combinación de "think" y "chunk", ya que la función se utiliza para pensar en una porción de código y evaluarla solo cuando sea necesario.



**Uso de Delay**

Se sabe que 'delayed' es una función en Dask que se usa para diferir la ejecución de funciones hasta que sea necesario. Se utiliza para paralelizar y distribuir la ejecución de tareas.



Ejemplo de un caso sencillo:

In [ ]:
from dask import delayed

@delayed
def add(a, b):
    return a + b

@delayed
def multiply(c, d):
    return c * d

x = add(1, 2)
y = add(3, 4)
z = multiply(x, y)
result = z.compute()
print(result)


## Parte 2: Generen varias funciones y construyan un grafo de paralelización con al menos 4 cuellos de botella.



El código genera un ticket de compra en el que se procesa el registro de venta.

In [ ]:
import dask.delayed as delayed
from dask.diagnostics import ProgressBar
import time
import numpy as np
import random

In [ ]:
class OrderLine():
  taxes = 0
  total = 0
  subtotal = 0
  type = 1

  def calculate_taxes(self):
    tax = 0.32 if self.type == 1 else 0.16
    self.taxes = self.subtotal * tax

  def round_taxes(self):
    self.taxes = round(self.taxes, 2)

  def calculate_total(self):
    self.total = self.subtotal + self.taxes


  def generar_ticket(nombre, total, items):
    print("¡Gracias por tu compra, {}!")
    print("Aquí está tu ticket:")
    print("---------------------")
    for item, precio in items.items():
        print("{}: ${}".format(subtotal))
    print("---------------------")
    print("Total: ${}".format(total))


In [ ]:
orders = []
for n in range(1):
  orders.append(OrderLine())

In [ ]:
def read_line():
  orders.append(OrderLine())

def calculate_taxes(order):
  order.calculate_taxes()

def round_taxes(order):
  order.round_taxes()

def calculate_total(order):
  order.calculate_total()

def create_ticket(order):
  order.generar_ticket()

In [ ]:
step0 = [delayed(read_line)() for i in range(5)]
total0 = delayed(sum)(step0)


step1 = [delayed(calculate_taxes)(j) for j in step0]
total = delayed(sum)(step1)

data2 = [delayed(round_taxes)(k, total) for k in orders]
total2 = delayed(sum)(data2)

data3 = [delayed(calculate_total)(l, total2) for l in orders]
total3 = delayed(sum)(data3)


data4 = [delayed(create_ticket)(m, total3) for m in orders]
total4 = delayed(sum)(data4)

total4.visualize()


# Extra challenge



> Buscar ensemble learning en Scikit learn, utilizar minimo 3 algoritmos de machine learning para crear un modelo paralelo basado en un ensamble. Generar uno con dask y otro con la libreria de ensemble de sklearn, y comparar resultados. 




In [ ]:
import pandas as pd
import time
from sklearn.metrics import accuracy_score
import dask
import dask.delayed
import time
import multiprocessing as mp


In [ ]:
df = pd.read_csv('/content/diabetes.csv')
df.head()

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from dask.distributed import Client
from sklearn.metrics import accuracy_score
import joblib
import timeit

import warnings
warnings.filterwarnings("ignore")

X = df.drop(columns = ['Outcome'])
y = df['Outcome']


#split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

# definir los clasificadores individuales
clf1 = LogisticRegression(random_state=42)
clf2 = DecisionTreeClassifier(random_state=42)
clf3 = SVC(probability=True, random_state=42)


# evaluar el rendimiento del clasificador de conjunto en el conjunto de prueba
y_pred = voting_clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Precisión del VotingClassifier: {:.2f}%".format(acc * 100))


**Dask**

In [ ]:
%time
client = Client()

# definir el clasificador de conjunto
voting_clf_dask = VotingClassifier(estimators=[('lr', clf1), ('dt', clf2), ('svc', clf3)], voting='hard')


with joblib.parallel_backend('dask'):
     voting_clf_dask.fit(X_train, y_train)

**Sklearn**

In [ ]:
%time
# definir el clasificador de conjunto
voting_clf_sklearn = VotingClassifier(estimators=[('lr', clf1), ('dt', clf2), ('svc', clf3)], voting='hard')
voting_clf_sklearn.fit(X_train, y_train)